In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from pydub import AudioSegment
from pydub.generators import Sine
import random
import numpy as np

# Define the note-to-frequency mapping for Do Major scale
note_frequencies = {
    'Do': 261.63,
    'Re': 293.66,
    'Mi': 329.63,
    'Fa': 349.23,
    'Sol': 392.00,
    'La': 440.00,
    'Si': 493.88,
    'Do_high': 523.25
}

# Set the duration for each note (in milliseconds)
note_duration = 100  # half a second per note

# Step 1: Create a 3-qubit quantum circuit
circuit = QuantumCircuit(3, 3)

# Step 2: Apply Hadamard gates to each qubit for superposition
circuit.h(0)
circuit.h(1)
circuit.h(2)

# Step 3: Measure each qubit
circuit.measure([0, 1, 2], [0, 1, 2])

# Step 4: Use the Aer simulator
simulator = Aer.get_backend('qasm_simulator')

# Transpile the circuit for the simulator
transpiled_circuit = transpile(circuit, simulator)

# Run the circuit with 1000 shots to generate many notes
job = simulator.run(transpiled_circuit, shots=1000)
result = job.result()
counts = result.get_counts()

# Define the state-to-note mapping based on measured states
note_mapping = {
    '000': 'Do', '001': 'Re', '010': 'Mi', '011': 'Fa',
    '100': 'Sol', '101': 'La', '110': 'Si', '111': 'Do_high'
}

# Initialize an empty audio segment to hold the music
audio = AudioSegment.silent(duration=0)

# Generate and append notes to the audio segment until we reach 1000 notes
note_count = 0
while note_count < 1000:
    for state, frequency in counts.items():
        # Continue only if we haven't reached 1000 notes yet
        if note_count >= 1000:
            break
        
        note = note_mapping[state]
        
        # Random chance of inserting a rest
        if random.random() < 0.2:  # 20% chance of rest
            audio += AudioSegment.silent(duration=note_duration)
        else:
            frequency = note_frequencies[note]
            sine_wave = Sine(frequency).to_audio_segment(duration=note_duration)
            # Apply fade in/out for smooth transitions
            sine_wave = sine_wave.fade_in(50).fade_out(50)
            audio += sine_wave
            
        note_count += 1

# Save the generated music to a new .wav file
audio.export("thousand_note_quantum_music.wav", format="wav")
print("Audio file 'thousand_note_quantum_music.wav' generated!")


Audio file 'thousand_note_quantum_music.wav' generated!
